In [1]:
from gurobipy import *

In [2]:
# Conjuntos: recursos e trabalhos
I = ['Carlos', 'Jonas', 'Monica']
J = ['Cientista', 'Desenvolvedor', 'Arquiteto']

In [3]:
key, perc = multidict({
    ('Carlos', 'Cientista') : 53,
    ('Carlos', 'Desenvolvedor') : 27,
    ('Carlos', 'Arquiteto') : 13,
    ('Jonas', 'Cientista') : 80,
    ('Jonas', 'Desenvolvedor') : 47,
    ('Jonas', 'Arquiteto') : 67,
    ('Monica', 'Cientista') : 53,
    ('Monica', 'Desenvolvedor') : 73,
    ('Monica', 'Arquiteto') : 47
})

In [4]:
m = Model('atrib')

Using license file C:\Users\mareg\gurobi.lic
Academic license - for non-commercial use only - expires 2021-01-25


In [5]:
# adicionando variaveis
x = m.addVars(key, vtype=GRB.BINARY, name='x')

In [6]:
# adicionando restrições
trabalho = m.addConstrs((x.sum('*', j) == 1 for j in J), 'trabalho')

In [8]:
# adicionando restrições de recurso
pessoa = m.addConstrs((x.sum(i, '*') <= 1 for i in I), 'pessoa')

In [9]:
m.setObjective(x.prod(perc), GRB.MAXIMIZE)

In [10]:
m.write('model/atrib.lp')

In [11]:
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xb6602fb2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6000000e+32   1.800000e+31   4.600000e+02      0s
       5    1.9300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds
Optimal objective  1.930000000e+02


In [13]:
# imprimindo solução
for var in m.getVars():
    if abs(var.x) > 1e-6:
        print(var.varName, '=', var.x)

x[Carlos,Cientista] = 1.0
x[Jonas,Arquiteto] = 1.0
x[Monica,Desenvolvedor] = 1.0


In [14]:
# Imprimindo valor objetivo
print('Percentual total = ', m.objVal)

Percentual total =  193.0


In [15]:
# alterando o cenario. Ana passa a fazer parte da seleção
nperc = {('Ana', 'Cientista'): 100,
         ('Ana', 'Desenvolvedor'): 100,
         ('Ana', 'Arquiteto'): 100
}

In [21]:
for key, val in nperc.items():
    i, j = key
    x[key] = m.addVar(obj=val, 
                      name=f'x[{i}, {j}]',
                      column=Column([1], [m.getConstrByName(f'trabalho[{j}]')]))

<gurobi.Column: 35.0 trabalho[Desenvolvedor] >